<a href="https://colab.research.google.com/github/abtheo/NaturalComputing/blob/main/Evolutionary_Computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

%pip install deap
import deap

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import random
random.seed(2021)
np.random.seed(2021)

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.14.3)


## Question 1. Schemata [0.5 point]

*Consider the two schemata:*

> $A1$ = #0#101###

> $A2$ = ##010#111

*Which of the two schemata has the highest chance to survive mutation, for a mutation rate $pm = 0.01$? (Justify your answer).*







### Answer 1. 
The **order** of a schema is determined by the number of fixed bits it contains, therefore: 

$o(A1) = 4$  
$o(A2) = 6$

The **defining length** of a schema is the difference between its last and first fixed string postition, therefore:


$d(A1) = (6 - 2) = 4$  
$d(A2) = (9 - 3) = 6$


A schema with higher defining length is easier to disrupt, and similarly a schema with higher order is easier to disrupt.


Schemata $A1$ has the highest chance to survive mutation because of its lower order and defining length. The specific rate of mutation is not relevant, because bits are selected for mutation via a random uniform distribution.


---



## Question 2. Building Block Hypothesis [0.5 point] 

*Describe a problem where the Building Block Hypothesis does not hold. Explain why.*


## Answer 2.

> *The Building Block Hypothesis (BBH) postulates that iteratively better candidate solutions can be generated by a stepwise process of selection, cross-over and mutation on low-order, high-fitness schemata.*  [Lecture 1 Slides]

BBH assumes modularity in the optimisation process, whereby each step taken results in some positive or negative impact on the fitness function. For example, in an easy-to-solve problem such as Counting Ones (see 4.2), each bit flip operation has a direct and measurable impact on the fitness of an individual. 

> *BBH does not hold when there is no information available which could guide a GA to the global optimum through the composition of sub-optimal solutions.*[Lecture 1 Slides]

The Needle-in-a-Haystack Problem is an example of a problem for which the BBH does not hold. The fitness function for the problem is defined as follows:

$$
f(x) =
  \begin{cases}
    1 & \text{if $x = s$}\\
    0 & \text{otherwise}\\
  \end{cases}
$$

In this case, exploring the search space grants zero information until the single correct condition is found. Thus, the genetic algorithm cannot be guided towards the global optimimum, and as the modularity assumption is broken. This concept is somewhat analagous to "Reward Sparsity" in Reinforcement Learning.


---




##Question 3. Selection Pressure [1 point] 

*Given the fitness function $f(x) = x^2$, calculate the
probability of selecting the individuals $x = 2$, $x = 3$, and $x = 4$, using roulette wheel selection.*



In [ ]:
X = np.array([2,3,4])
fX = X**2
fX_wheel = np.divide(1, sum(fX)) * fX

px.pie(names=["Where x=2","Where x=3","Where x=4"], values = fX_wheel, title=f"Roulette Wheel of Probabilities: ƒ(x) = x^2").show()

*Calculate the probability of selecting the same individuals when the fitness function is scaled as follows $f1(x) = f(x) + 20$.*


In [ ]:
X = np.array([2,3,4])
f1 = X**2 + 20
f1_wheel = np.divide(1, sum(f1)) * f1

px.pie(names=["Where x=2","Where x=3","Where x=4"], values = f1_wheel, title=f"Roulette Wheel of Probabilities: ƒ1(x) = ƒ(x) + 20")


*Which fitness function yields a lower selection pressure? What can you conclude about the effect of fitness scaling
on selection pressure?*

### Answer 3.


$f1(x)$ has a lower selection pressure than $f(x)$. The likelihood of selecting the highest fitness candidates is 40.4% for $f1(x)$, which is a lower proportion than the 55.2% of $f(x)$.

Having too high a likelihood for selecting only the fittest individuals is *Greedy*, whereby the exploration/exploitation trade-off is heavily biased towards exploitation. 

Such a situation compounds over each generation, which drives extreme selection pressure, quickly leading to a population of *Super-Fit* individuals. This is undesirable as it eliminates the opportunity to explore much of the search space, and may force the algorithm to converge to a suboptimal local minima/maxima.


---



# 4.1. Role of selection in GA’ s [2 points] 

*A simple (1 + 1)-GA for binary problems works as follows.*

>* (a) Randomly generate a bit sequence $x$.
* (b) Create a copy of x and invert each of its bits with probability $p$. Let $xm$ be the result.
* (c) If $xm$ is closer to the goal sequence than $x$ then replace $x$ with $xm$.
* (d) Repeat the process from step (b) with the new $x$ until the goal sequence is reached.


# 4.2 Counting Ones 
*The Counting Ones problem amounts to find a bit string whose sum of its entries is
maximum. Implement a simple (1 + 1)-GA for solving the Counting Ones problem.*

> * (a) Use bit strings of length $l = 100$ and a mutation rate $p = 1/l$. For a run of 1500
iterations, plot the best fitness against the elapsed number of iterations.


In [ ]:
L = 100
p = np.divide(1, L)
x = np.random.randint(2, size=(L,))
P = np.random.random(size=(L,))

#Binary threshold on probability p,
#Sum with X and modulo to flip appropriate bits
xm = (x + (P<p)) % 2


* (b) Now do 10 runs. How many times does the algorithm find the optimum?

In [ ]:
#4.2.a & 4.2.b) Plot best fitness over iterations
best_fitness_array = []
for run in range(10):
  #Can't perform max on an empty array, have to initialise
  best_fitness = [0]
  for i in range(1500):
    x = np.random.randint(2, size=(L,))
    #Keep highest fitness
    max_x = max(sum(x), max(best_fitness))
    best_fitness.append(max_x)
  #Cheeky hack to remove the initialised value at index 0
  best_fitness_array.append(best_fitness[1:])

#Descriptive statistics for the *last iteration only*
best_fitness_np = np.array(best_fitness_array)
print(f"Mean: {np.mean(best_fitness_np[:,-1])},  std: {np.std(best_fitness_np[:,-1])}")

#Plot evolution runs
px.line(y=best_fitness_array, title="Best Fitness for Random Bitstrings")

Mean: 67.0,  std: 1.4832396974191326


* (c) Now replace (c) in the above algorithm with (c’): replace $x$ with $xm$. Is there a difference in performance when using this modification? Justify your answer.

In [ ]:
#4.2.c) Plot best fitness over iterations *with mutation*
best_fitness_array = []
for run in range(10):
  best_fitness = [0]

  x = np.random.randint(2, size=(L,))
  for i in range(1500):
    
    #Mutate
    P = np.random.random(size=(L,))
    xm = (x + (P<p)) % 2

    #Replace x with mutation if fitness is higher
    if sum(xm) > sum(x):
      x = xm

    max_x = max(sum(xm), max(best_fitness))
    best_fitness.append(max_x)
  #Cheeky hack to remove the initialised value at index 0
  best_fitness_array.append(best_fitness[1:])


#Descriptive statistics for the *last iteration only*
best_fitness_np = np.array(best_fitness_array)
print(f"Mean: {np.mean(best_fitness_np[:,-1])},  std: {np.std(best_fitness_np[:,-1])}")

#Plot evolution runs
px.line(y=best_fitness_array, title="Best Fitness for (1+1)-GA algorithm").show()


Mean: 99.8,  std: 0.4



## Answer 4.

When randomly generating bit strings, the best fitness of the population is extremely unlikely to reach optimum. The highest observed in fitness in the plot above is typically around ~70. In my example above, the distribution of the final epoch of each run has a mean of 66.2 and a standard deviation of 1.4.


After implementing the (1 + 1)-GA algorithm, the results are greatly improved. On every single run, the algorithm achieved a best fitness of 100, meaning the distribution of the final epoch has a mean of 100 and variance 0. This method also clearly converges faster than random generation.


---





## Question 5. Evolutionary strategies vs local search [1 point] 

*Consider a (1+5) ES. How does this differ from the (1+1) ES in how the search space is explored when optimizing a function? How does the (1+λ) ES strategy behave with respect to the value of
λ when compared to greedy algorithms? (Recall that greedy algorithms perform a
sequence of locally optimal steps in order to search for an optimal solution.)*






## Answer 5.
The parameter λ refers to the number of offspring generated by the population at each iteration. Therefore, a (1+1) ES produces only a fifth as many children per iteration as a (1+5) ES. Having more children increases the amount of search space able to be explored by the population in each epoch, due to having more possibility of an individual mutating in any given optimisation direction. 


Increasing the value of λ generally leads to a less greedy algorithm. The parents in each iteration are typically selected with a uniform random distribution, so increasing λ lowers selection pressure resulting in longer convergence. 

---

##Question 6. Memetic algorithms vs simple EAs [2.5 point] 


*Implement the simple EA for the TSP described in our first lecture (see slides)*.




*Implement a variant of this algorithm based on memetic algorithms. Compare
the performance of the two algorithms in a fair way on the TSP problem instance given in the file ‘file-tsp’ and on one small instance at your choice from
the ‘Symmetric Traveling Salesman Problem’ benchmark instances available at
http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html.*

*The file
‘file-tsp’ contains a 50 × 2 matrix with the coordinates $(x_i, y_i)$ for city $i =1, . . . , 50.$*

In [ ]:
##I didn't wanna mess around with filesharing for reproducability.
##Read the city matrix from a Base64 encoded string.
import base64
base64_matrix_string = "MC4yNTU0LDE4LjIzNjYNCjAuNDMzOSwxNS4yNDc2DQowLjczNzcsOC4zMTM3DQoxLjEzNTQsMTYuNTYzOA0KMS41ODIwLDE3LjMwMzANCjIuMDkxMyw5LjI5MjQNCjIuMjYzMSwxNy4zMzkyDQoyLjYzNzMsMi42NDI1DQozLjAwNDAsMTkuNTcxMg0KMy42Njg0LDE0LjgwMTgNCjMuODYzMCwxMy43MDA4DQo0LjIwNjUsOS44MjI0DQo0LjgzNTMsMi4wOTQ0DQo0Ljk3ODUsMy4xNTk2DQo1LjM3NTQsMTcuNjM4MQ0KNS45NDI1LDYuMDM2MA0KNi4xNDUxLDMuODEzMg0KNi43NzgyLDExLjAxMjUNCjYuOTIyMyw3Ljc4MTkNCjcuNTY5MSwwLjkzNzgNCjcuODE5MCwxMy4xNjk3DQo4LjMzMzIsNS45MTYxDQo4LjU4NzIsNy44MzAzDQo5LjEyMjQsMTQuNTg4OQ0KOS40MDc2LDkuNzE2Ng0KOS43MjA4LDguMTE1NA0KMTAuMTY2MiwxOS4xNzA1DQoxMC43Mzg3LDIuMDA5MA0KMTAuOTM1NCw1LjE4MTMNCjExLjM3MDcsNy4yNDA2DQoxMS43NDE4LDEzLjY4NzQNCjEyLjA1MjYsNC43MTg2DQoxMi42Mzg1LDEyLjEwMDANCjEzLjA5NTAsMTMuNjk1Ng0KMTMuMzUzMywxNy4zNTI0DQoxMy44Nzk0LDMuOTQ3OQ0KMTQuMjY3NCwxNS44NjUxDQoxNC41NTIwLDE3LjI0ODkNCjE0Ljk3MzcsMTMuMjI0NQ0KMTUuMjg0MSwxLjQ0NTUNCjE1LjU3NjEsMTIuMTI3MA0KMTYuMTMxMywxNC4yMDI5DQoxNi40Mzg4LDE2LjAwODQNCjE2Ljc4MjEsOS40MzM0DQoxNy4zOTI4LDEyLjk2OTINCjE3LjUxMzksNi40ODI4DQoxNy45NDg3LDcuNTU2Mw0KMTguMzk1OCwxOS41MTEyDQoxOC45Njk2LDE5LjM1NjUNCjE5LjA5MjgsMTYuNTQ1Mw=="
text_file = base64.b64decode(base64_matrix_string).decode('utf-8')

tsp = np.array([np.array(x.split(','),dtype=float) for x in text_file.splitlines()])
print(tsp.shape)

px.scatter(x=tsp[:,0], y=tsp[:,1], color_discrete_sequence=["#FF0000"])

(50, 2)


In [ ]:

#Evaluate fitness of each candidate -
#Total Euclidean distance of the ordered path
def fitness(candidate, tsp):
  return sum([np.linalg.norm(tsp[candidate][i] - tsp[candidate][i-1]) \
     for i in range(1, len(candidate))])
  

def tournament_select(population,tsp, K=2):
  #Select K candidates at random
  candidate_parents = np.random.randint(len(population),size=K)
  #Calculate fitness of candidates
  candidate_fitness = [fitness(population[p],tsp) for p in candidate_parents]
  #Return the fitest candidate as the winning parent (lowest in this case)
  return population[candidate_parents[np.argmax(candidate_fitness)]]

def wrap_fill(parent, child, cross_points):
  #Wrap-around parent starting at right-hand-side
  wrap_parent =  list(parent[cross_points[1]:]) + \
                    list(parent[:cross_points[1]])
  #Same for indexes
  index_range = list(range(cross_points[1], parent.shape[0])) + list(range(0, cross_points[1]))

  for j in index_range:
    for k in wrap_parent:
      if k not in child:
        child[j] = k

  return child

#Reverse Sequence Mutation
def mutate(child, mu=0.05):
  cross_points = np.random.randint(child.shape[0]-1, size=2)
  cross_points.sort()
  #Ensure min gap length of 1
  cross_points[1] += 1

  #Swap values between cross-over points
  if np.random.random() <= mu:
    child[cross_points[0]], child[cross_points[1]] = child[cross_points[1]] , child[cross_points[0]] 

  return child

def order_crossover(mum, dad):
  cross_points = np.random.randint(mum.shape[0]-1, size=2)
  cross_points.sort()
  #Ensure min gap length of 1
  cross_points[1] += 1

  #Copy selection to offspring
  son = np.zeros_like(mum)
  son[cross_points[0]:cross_points[1]] = mum[cross_points[0]:cross_points[1]]

  
  daughter = np.zeros_like(dad)
  daughter[cross_points[0]:cross_points[1]] = dad[cross_points[0]:cross_points[1]]

  
  #Fill missing cities from the other parent
  son = wrap_fill(dad, son, cross_points)
  daughter = wrap_fill(mum, daughter, cross_points)

  return [son, daughter]


  

In [ ]:
def evolutionary_algorithm(tsp, elitism=False, p_size=100, generations=100):
  #Initialize the population with Candidate Solutions
  population = np.array([random.sample(range(tsp.shape[0]), tsp.shape[0]) for _ in range(p_size)])
  print("Population Shape:", population.shape)

  generation_log = []
  #Loop:
  for i in tqdm(range(generations)):
    #Select parents for reproduction -
    #Tournament Selection method with replacement
    parents = np.array([tournament_select(population,tsp) for p in range(p_size)])

    #Recombination of parents
    children = np.array(
        [order_crossover(parents[i-1], parents[i]) for i in range(len(parents))]
      )
    children = children.reshape( (children.shape[0]*children.shape[1], children.shape[2]) )
    #Mutate children
    candidates = np.array([mutate(child) for child in children])

    #ES(1+λ) = Elitism True, as opposed to (1,λ) for False
    if elitism:
      candidates = np.concatenate( (candidates, parents) )

    #Evaluate children
    candidates_fitness = np.array([fitness(x,tsp) for x in candidates])

    #Select next generation -
    #Deterministically cut out the worst fitness individuals
    population = candidates[np.argpartition(candidates_fitness, p_size)[:p_size]]

    generation_log.append({"high":np.max(candidates_fitness),
                          "low":np.min(candidates_fitness),
                          "open": np.percentile(candidates_fitness,75),
                          "close": np.percentile(candidates_fitness,25),
                          "best": candidates[np.argmin(candidates_fitness)]
                          })
    
  return generation_log


### Note on Generation Log:

Open and Close correspond to the 1st and 4th Quartiles of Fitness per generation. They are named as such to allow convenient plotting in Candlestick form, as seen below.



In [ ]:
generation_log_tsp = evolutionary_algorithm(tsp, elitism=True)

df_tsp  = pd.DataFrame(generation_log_tsp)
fig = go.Figure(
        data=[go.Candlestick(
                x=df_tsp.index,
                open=df_tsp['open'],
                high=df_tsp['high'],
                low=df_tsp['low'],
                close=df_tsp['close']
              )],
          layout={
              "title": "Population Fitness over Generations",
              "xaxis": {"title":"Generation"},
              "yaxis": {"title":"Fitness"},
              }
        ).show()

best = df_tsp['best'][np.argmin(df_tsp['low'])]
print(f"Best candidate: {best}\nWith a fitness of {fitness(best,tsp)}")

Population Shape: (100, 50)


Best candidate: [23  9 45 27 28 22 21 13 39 46 38 11  2  5 20 18  7 12 29 24 34 33 32 15
 16 31 26 37 30  6  3  4  0 19 40 41 47 49 43 36 25 35 44 42 48 17 10 14
  1  8]
With a fitness of 308.1715196296436


##TSP: 48 Capitals of the US

One of the Symmetric traveling salesman problems (TSP). Dataset found at http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp . Optimal path found at http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.opt.tour

In [ ]:
#Same trick again, load data from string
us_capitals_b64str = "MTM0LDE0NTMKMjMzLDEwCjUzMCwxNDI0CjAxLDg0MQowODIsMTY0NAo2MDgsNDQ1OAo1NzMsMzcxNgoyNjUsMTI2OAo4OTgsMTg4NQoxMTEyLDIwNDkKNTQ2OCwyNjA2CjU5ODksMjg3Mwo0NzA2LDI2NzQKNDYxMiwyMDM1CjYzNDcsMjY4Mwo2MTA3LDY2OQo3NjExLDUxODQKNzQ2MiwzNTkwCjc3MzIsNDcyMwo1OTAwLDM1NjEKNDQ4MywzMzY5CjYxMDEsMTExMAo1MTk5LDIxODIKMTYzMywyODA5CjQzMDcsMjMyMgo2NzUsMTAwNgo3NTU1LDQ4MTkKNzU0MSwzOTgxCjMxNzcsNzU2CjczNTIsNDUwNgo3NTQ1LDI4MDEKMzI0NSwzMzA1CjY0MjYsMzE3Mwo0NjA4LDExOTgKMjMsMjIxNgo3MjQ4LDM3NzkKNzc2Miw0NTk1CjczOTIsMjI0NAozNDg0LDI4MjkKNjI3MSwyMTM1CjQ5ODUsMTQwCjE5MTYsMTU2OQo3MjgwLDQ4OTkKNzUwOSwzMjM5CjEwLDI2NzYKNjgwNywyOTkzCjUxODUsMzI1OAozMDIzLDE5NDI="
text_file = base64.b64decode(us_capitals_b64str).decode('utf-8')

tsp_us = np.array([np.array(x.split(','),dtype=int) for x in text_file.splitlines()])
print(tsp_us.shape)


px.scatter(x=tsp_us[:,0], y=tsp_us[:,1], color_discrete_sequence=["#FF0000"])

(48, 2)


In [ ]:
optimal_path = np.array([1,8,38,31,44,18,7,28,6,37,19,27,17,43,30,36,46,33,20,47,21,32,39,48,5,42,24,10,45,35,4,26,2,29,34,41,16,22,3,23,14,25,13,11,12,15,40,9]) -1
print(f"Optimal path fitness: {fitness(optimal_path,tsp_us)}")

generation_log_tsp_us = evolutionary_algorithm(tsp_us, elitism=True)

df_tsp_us  = pd.DataFrame(generation_log_tsp_us)
fig = go.Figure(
        data=[go.Candlestick(
                x=df_tsp_us.index,
                open=df_tsp_us['open'],
                high=df_tsp_us['high'],
                low=df_tsp_us['low'],
                close=df_tsp_us['close']
              )],
          layout={
              "title": "Population Fitness over Generations",
              "xaxis": {"title":"Generation"},
              "yaxis": {"title":"Fitness"},
              }
        ).show()

best_us = df_tsp_us['best'][np.argmin(df_tsp_us['low'])]
print(f"Best candidate: {best_us}\nWith a fitness of {fitness(best_us,tsp_us)}")

Optimal path fitness: 84032.55841236569
Population Shape: (100, 48)


Best candidate: [19 11 10 12 31 41  9 32 40 28 47 37 45 27 14 30 15 39 16 35 43 42 36 18
 26 38  7 34  2 25  1  3 44 22 23  4  0 20 46 13 21 33 17 29 24  5  6  8]
With a fitness of 90746.61723922033


## Question 6. Memetic Algorithm

Using a manual implementation of a Local Search algorithm, basically just brute-force local mutations by swapping pairs of points. "Recent Advances in Memetic Algorithms" (2004) used as reference material.

In [ ]:
"""
Custom implementation of a Local Search:
* Sequentially swaps each pair of chromosomes in an individual (respectful node exchange operators)
* When a mutant is found which has lower fitness, stop the search (greedy)
"""
def local_search(population, tsp):
  #(100,50)
  local_optima = []
  for individual in population:
    #(50,)
    #Calculate fitness of original individual
    i_fitness = fitness(individual, tsp)
    improved_flag = False
    for i in range(2,len(individual)):
      mutant_child = individual.copy()
      #Swap each pair of chromosomes sequentially
      mutant_child[i-2:i] = mutant_child[i-2:i][::-1]
      #Recalculate fitness
      m_fitness = fitness(mutant_child, tsp)
      #Greedily keep the first improvement we find (want to cut down on runtime)
      if m_fitness < i_fitness:
        local_optima.append(mutant_child)
        improved_flag = True
        break
    #If no local improvement found, keep original
    if not improved_flag:
      local_optima.append(individual)

  return np.array(local_optima)

def memetic_algorithm(tsp, elitism=False, p_size=100, generations=100):
  #Initialize the population with Candidate Solutions
  population = np.array([random.sample(range(tsp.shape[0]), tsp.shape[0]) for _ in range(p_size)])
  print("Population Shape:", population.shape)
  print("Initial Fitness: ", sum([fitness(x, tsp) for x in population]))

  #Apply LOCAL SEARCH to each individual
  population = local_search(population, tsp)
  print("Fitness after Local Search: ", sum([fitness(x, tsp) for x in population]))

  generation_log = []
  #Loop:
  for i in tqdm(range(generations)):
    #Select parents for reproduction -
    #Tournament Selection method with replacement
    parents = np.array([tournament_select(population,tsp) for p in range(p_size)])

    #Recombination of parents
    children = np.array(
        [order_crossover(parents[i-1], parents[i]) for i in range(len(parents))]
      )
    children = children.reshape( (children.shape[0]*children.shape[1], children.shape[2]) )
    #Mutate children
    candidates = np.array([mutate(child) for child in children])
    #ES(1+λ) = Elitism True, as opposed to (1,λ) for False
    if elitism:
      candidates = np.concatenate( (candidates, parents) )

    #Apply LOCAL SEARCH to each individual
    candidates = local_search(candidates, tsp)

    #Evaluate children
    candidates_fitness = np.array([fitness(x,tsp) for x in candidates])

    #Select next generation -
    #Deterministically cut out the worst fitness individuals
    population = candidates[np.argpartition(candidates_fitness, p_size)[:p_size]]

    generation_log.append({"high":np.max(candidates_fitness),
                          "low":np.min(candidates_fitness),
                          "open": np.percentile(candidates_fitness,75),
                          "close": np.percentile(candidates_fitness,25),
                          "best": candidates[np.argmin(candidates_fitness)]
                          })
    
  return generation_log

In [ ]:
generation_log_memetic = memetic_algorithm(tsp, elitism=True)

df_memetic_tsp  = pd.DataFrame(generation_log_memetic)
fig = go.Figure(
        data=[go.Candlestick(
                x=df_memetic_tsp.index,
                open=df_memetic_tsp['open'],
                high=df_memetic_tsp['high'],
                low=df_memetic_tsp['low'],
                close=df_memetic_tsp['close']
              )],
          layout={
              "title": "Population Fitness over Generations",
              "xaxis": {"title":"Generation"},
              "yaxis": {"title":"Fitness"},
              }
        ).show()

best_mem = df_memetic_tsp['best'][np.argmin(df_memetic_tsp['low'])]
print(f"Best candidate: {best_mem}\nWith a fitness of {fitness(best_mem,tsp)}")

Population Shape: (100, 50)
Initial Fitness:  49952.51372114572
Fitness after Local Search:  49477.96084808358


Best candidate: [39 33 20 11  3  5 32 16  7 27 29 40 24  1 14 23 17 10 26 37 36 41 30 38
 48 44 46 15 13 28 49 42 34  4  0  6  8  9  2 12 31 45 25 22 18 21 19 35
 43 47]
With a fitness of 296.00062391686026


In [ ]:
generation_log_memetic_us = memetic_algorithm(tsp_us, elitism=True)

df_memetic_tsp_us  = pd.DataFrame(generation_log_memetic_us)
fig = go.Figure(
        data=[go.Candlestick(
                x=df_memetic_tsp_us.index,
                open=df_memetic_tsp_us['open'],
                high=df_memetic_tsp_us['high'],
                low=df_memetic_tsp_us['low'],
                close=df_memetic_tsp_us['close']
              )],
          layout={
              "title": "Population Fitness over Generations",
              "xaxis": {"title":"Generation"},
              "yaxis": {"title":"Fitness"},
              }
        ).show()

best_mem_us = df_memetic_tsp_us['best'][np.argmin(df_memetic_tsp_us['low'])]
print(f"Best candidate: {best_mem_us}\nWith a fitness of {fitness(best_mem_us,tsp_us)}")

Population Shape: (100, 48)
Initial Fitness:  17754639.53361642
Fitness after Local Search:  17565559.39685175


Best candidate: [43 17 26 19 29 18 36 30 39 16 27 37 15 20 32 42 35 45 31 28  4 34  3  8
  9  5  0  1 25 41 38 40 33 13 24  7 47  6 23 44  2 46 14 21 22 12 10 11]
With a fitness of 82148.30641616145


## Question 6.b. Discussion
*On the TSP problem are memetic algorithms more effective than the simple
EA’ s? (To answer this question, use the results of your investigation as well as recent results from the literature).*

### Answer 6.b.

The literature clearly supports the use of memetic algorithms for tackling the Traveling Salesman Problem (TSP) in the paper "Memetic Algorithms for the Traveling Salesman Problem" by P. Merz & B. Freisleben. Implementations of memetic algorithms with local search were shown to be at least marginal improvements over EAs on a variety of TSP problems.


The outcomes of my investigation are positive! Despite using a rather rudimentary 2-length swap as my local search, the memetic algorithm converged to a better 'best fitness' individual in both the provided TSP dataset and the US Cities TSP. Convergence also appears to take fewer iterations.

Keeping parents in each generation with an Elitist strategy ES(1+λ) provided vastly superior results as opposed to (1,λ), in the case of both the 'vanilla' EA and the Memetic Algorithm.








---


##Question 7. Genetic Programming representation [0.5 point] 

*Give a suitable function, terminal set and s-expression for the following logical and mathematical formulas:*

> a.  $(y ∧ true) → ((x ∨ y) ∨ (z ↔ (x ∧ y)))$

> b. $0.234 ∗ z + x – 0.789$

### Answer 7.

a.
* Function Set: $[∧, ∨, ↔]$
* Terminal Set: $[x, y, z, true]$
* S-Expression: $[∧, ∨, ↔, x, y, z, true]$

b. 
* Function Set: $[*, +, -]$
* Terminal Set: $[x, z, 0.234, 0.789]$
* S-Expression: $[*, +, -, x, z, 0.234, 0.789]$
---



## Question 8. 8. Genetic Programming behaviour [2 points] 

*Implement a GP program for finding a symbolic expression that fits the following data:*


In [ ]:
data = np.array([
  [-1.0,0.0000],
  [-0.9,-0.1629],
  [-0.8,-0.2624],
  [-0.7,-0.3129],
  [-0.6,-0.3264],
  [-0.5,-0.3125],
  [-0.4,-0.2784],
  [-0.3,-0.2289],
  [-0.2,-0.1664],
  [-0.1,-0.0909],
  [0,0.0],
  [0.1,0.1111],
  [0.2,0.2496],
  [0.3,0.4251],
  [0.4,0.6496],
  [0.5,0.9375],
  [0.6,1.3056],
  [0.7,1.7731],
  [0.8,2.3616],
  [0.9,3.0951],
  [1.0,4.0000]
])


Credit to the DEAP documentation @ https://deap.readthedocs.io/en/master/examples/gp_symbreg.html

In [ ]:
creator = None
from deap import creator, base, tools, algorithms, gp
import operator

pset = gp.PrimitiveSet("MAIN", 1)
#function_set = {+, −, ∗, log, exp, sin, cos, div}
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

def safe_log(x):
  try:
      return np.log(np.abs(x+1e-4))
  except:
      return 1

def safe_exp(x):
  try:
      return np.exp(x)
  except:
      return 1

pset.addPrimitive(safe_log, 1)
pset.addPrimitive(safe_exp, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(np.cos, 1)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
y = data[:,1]
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    #Make 'predictions'
    try:
      yhat = list([func(x) for x in points])
    except:
      return 1,
    #sum of absolute errors
    return np.sum(np.abs(y - np.array(yhat))),

toolbox.register("evaluate", evalSymbReg, points=data[:,0])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [ ]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [ ]:
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
# number of generations 50,
# crossover probability 0.7,
# mutation probability: 0
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0, ngen=50, stats=mstats,
                                halloffame=hof, verbose=True)

   	      	                    fitness                    	                     size                    
   	      	-----------------------------------------------	---------------------------------------------
gen	nevals	avg    	gen	max    	min    	nevals	std    	avg    	gen	max	min	nevals	std  
0  	300   	23.5898	0  	206.184	10.2331	300   	18.6357	3.20333	0  	7  	2  	300   	1.284
1  	216   	15.7492	1  	57.6894	4.24698	216   	4.99074	3.03   	1  	7  	2  	216   	1.16437
2  	216   	15.1509	2  	78.1686	4.24698	216   	7.30721	2.98   	2  	7  	2  	216   	1.15741
3  	206   	15.2379	3  	206.184	4.24698	206   	13.1907	3.05   	3  	8  	2  	206   	1.3016 
4  	222   	15.5467	4  	98.4403	4.24698	222   	10.3031	3.71333	4  	9  	2  	222   	1.6014 
5  	202   	13.5186	5  	40.7796	2.3637 	202   	8.68224	4.24   	5  	9  	2  	202   	1.59867
6  	220   	12.8288	6  	98.4403	2.3637 	220   	11.9643	4.57   	6  	10 	2  	220   	1.49168
7  	214   	11.4991	7  	40.7796	2.3637 	214   	10.3345	4.49333	7  	9  	2  	214   	1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:202: RuntimeWarning:

invalid value encountered in subtract



12 	226   	2.58829e+15	12 	7.76487e+17	2.3637 	226   	4.47557e+16	5.52667	12 	12 	2  	226   	2.12508
13 	204   	12751.5    	13 	3.82217e+06	2.3637 	204   	220305     	5.92667	13 	13 	2  	204   	2.25713
14 	208   	9.57309    	14 	94.7729    	2.3637 	208   	11.1905    	6.65333	14 	13 	2  	208   	2.45081
15 	210   	8.74594    	15 	81.0689    	2.3637 	210   	8.92288    	7.32667	15 	13 	2  	210   	2.31228
16 	218   	8.11785    	16 	81.0689    	2.3637 	218   	8.3681     	7.74   	16 	15 	2  	218   	2.43565
17 	218   	7.62365    	17 	83.5115    	2.3637 	218   	9.01378    	7.73   	17 	15 	2  	218   	2.37566
18 	194   	12.8324    	18 	1842.82    	2.3637 	194   	106.15     	7.89   	18 	15 	2  	194   	2.01938
19 	186   	5.94784    	19 	32.3798    	2.3637 	186   	6.15633    	8.06   	19 	17 	3  	186   	1.98572
20 	206   	6.59959    	20 	32.3798    	2.3637 	206   	6.34652    	7.97333	20 	13 	3  	206   	2.00482
21 	220   	6.6681     	21 	32.3798    	2.3637 	220   	6.39148    	7.91333	21 	14 	3  	220  

In [ ]:
px.line(y=log.chapters["fitness"].select("min"), 
        labels={"x":"Generation", "y":"Sum of Absolute Error"},
        title="Fitness per Generation")

In [ ]:
px.line(y=log.chapters["size"].select("max"), 
        labels={"x":"Generation", "y":"Max Tree Size"},
        title="Maximum Tree Size per Generation",
        color_discrete_sequence=['#ff0000'])

### Answer 8.

Yes, this is a classic case of **overfitting due to bloat**.



To combat this phenomenon, we can place hard limits on the size and depth of our trees. Other possible methods are size fair crossover and size fair mutation, whereby potential mutation and crossover updates are constrained by the size of subtrees under consideration. Finally, we could bias the selection method against choosing larger trees with something like the Tarpeian method.


http://www0.cs.ucl.ac.uk/staff/W.Langdon/ftp/papers/poli08_fieldguide.pdf


